In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import time

from torchvision import transforms
from torchvision.datasets import MNIST # example1 data
from torch.autograd import Variable

from torch.utils.data import DataLoader 
#batch, shuffle and load data in parrallel
from torch.utils.data.sampler import SubsetRandomSampler 
#partition data into training and validation sets

from torch import optim
#used with loss function, defines learning rate, momentum...

# Fundamentals of Neural Net
## First define layers

In [ ]:
# we will construct a nn with 5 input nodes, 
#1 hidden layer with 3 noeds, and 1 output layer
n_input, n_hidden, n_output = 5, 3, 1

# parameter initialization
weigths and bias parameters for each layer as tensors(matrice)

In [ ]:
# tensors for input and outputs
x = torch.randn((1, n_input))
y = torch.randn((1, n_output))
print(x)
print(y)

In [ ]:
# weights (y,x) :
w1 = torch.randn(n_input, n_hidden) # weight for hidden layer
w2 = torch.randn(n_hidden, n_output) # weight for output layer

print(w1)
print(w2)

In [ ]:
# bias terms for each layer
b1 = torch.randn((1, n_hidden)) # bias for hidden layer
b2 = torch.randn((1, n_output)) # bias for output layer
print(b1)
print(b2)

# 4 steps to train neural net:

    Forward Propagation
    Loss computation
    Backpropagation
    Updating the parameters


# Forward Propagation
## computing activations for layers

    z = weight * input + bias
    a = activation_function (z)


In [ ]:
# using sigmoid
def sigmoid_activation(z):
    return 1 / (1 + torch.exp(-z))

In [ ]:
# activation of hidden layer 
z1 = torch.mm(x, w1) + b1    # x - input, w1 - weight, b1 - bias
a1 = sigmoid_activation(z1)

In [ ]:
# activation (output) of final layer 
z2 = torch.mm(a1, w2) + b2      #activation of prev layer taken as input
output = sigmoid_activation(z2)

# Loss computation
## defining loss function

In [ ]:
# how far output is from deired value
loss = y - output

# Back Propagation
## recompute parameters by using derivative of error term

In [ ]:
## function to calculate the derivative of activation
def sigmoid_delta(x):
    return x * (1 - x)

In [ ]:
## compute derivative of error terms
delta_output = sigmoid_delta(output)
delta_hidden = sigmoid_delta(a1)

In [ ]:
## backpass the changes to previous layers 
d_outp = loss * delta_output
loss_h = torch.mm(d_outp, w2.t())
d_hidn = loss_h * delta_hidden

# Update Parameters
## specifying learning rate and assigning new weights and biases

In [ ]:
learning_rate = 0.1

In [ ]:
w2 += torch.mm(a1.t(), d_outp) * learning_rate
w1 += torch.mm(x.t(), d_hidn) * learning_rate

In [ ]:
b2 += d_outp.sum() * learning_rate
b1 += d_hidn.sum() * learning_rate

# Example 1
## Defining Neural Net

In [53]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 600)
        self.fc2 = nn.Linear(600, 400)
        self.fc3 = nn.Linear(400, 200)
        self.fc4 = nn.Linear(200, 10)

        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x)) 
        return x

In [ ]:
F.log_softmax?

In [54]:
#create instance
net = Net()

## loading data

In [5]:
batch_size = 200
train_loader = torch.utils.data.DataLoader(MNIST('../data', train=True, download=True,transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])),batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(MNIST('../data', train=False, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])),batch_size=batch_size, shuffle=True)

## Specify optimiser and loss function

In [56]:
# create a stochastic gradient descent optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
# create a loss function
criterion = nn.NLLLoss()

# Run training

In [7]:
print(train_loader[1])

TypeError: 'DataLoader' object does not support indexing

In [58]:
log_interval = 100
start = time.time()

for epoch in range(1, 10): ## run the model for 10 epochs

    for batch_idx, (data, target) in enumerate(train_loader):
       
        data, target = Variable(data), Variable(target)
        
        # resize data from (batch_size, 1, 28, 28) to (batch_size, 28*28)
        data = data.view(-1, 28*28)
        #print(data.size())
        optimizer.zero_grad()
        
        ## 1. forward propagation
        net_out = net(data)
        
        ## 2. loss calculation
        loss = criterion(net_out, target)
        
        ## 3. backward propagation
        loss.backward()
        
        ## 4. weight optimization
        optimizer.step()
        

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, 
                batch_idx * len(data), 
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader), 
                loss.data))
            
end = time.time()
print(end-start)

/home/werner/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301998
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.491649
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.333008
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.203353
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.215467
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.172783
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.194003
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.132691
Train Epoch: 3 [40000/60000 (67%)]	Loss: 0.132637
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.115938
Train Epoch: 4 [20000/60000 (33%)]	Loss: 0.101211
Train Epoch: 4 [40000/60000 (67%)]	Loss: 0.054802
Train Epoch: 5 [0/60000 (0%)]	Loss: 0.046972
Train Epoch: 5 [20000/60000 (33%)]	Loss: 0.063006
Train Epoch: 5 [40000/60000 (67%)]	Loss: 0.069938
Train Epoch: 6 [0/60000 (0%)]	Loss: 0.091267
Train Epoch: 6 [20000/60000 (33%)]	Loss: 0.052344
Train Epoch: 6 [40000/60000 (67%)]	Loss: 0.081144
Train Epoch: 7 [0/60000 (0%)]	Loss: 0.041821
Train Epoch: 7 [20000/60000 (33%)]	Loss: 0.023665
Train Epoch: 7 [40000/60000 (67%)]	

In [60]:
# run a test loop
test_loss = 0
correct = 0
for data, target in test_loader:
    data, target = Variable(data), Variable(target)
    data = data.view(-1, 28 * 28)
    net_out = net(data)
    # sum up batch loss
    test_loss += criterion(net_out, target).data
    pred = net_out.data.max(1)[1]  # get the index of the max log-probability
    correct += pred.eq(target.data).sum()

test_loss /= len(test_loader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, 
    correct, 
    len(test_loader.dataset),
    100. * correct / len(test_loader.dataset))
     )

/home/werner/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  



Test set: Average loss: 0.0003, Accuracy: 9791/10000 (97%)



Prev Test set: Average loss: 0.0004, Accuracy: 9783/10000 (97%)

# Cuda implementation

In [37]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

net2 = Net()
net2.to(device)

cuda:0


Net(
  (fc1): Linear(in_features=784, out_features=600, bias=True)
  (fc2): Linear(in_features=600, out_features=400, bias=True)
  (fc3): Linear(in_features=400, out_features=200, bias=True)
  (fc4): Linear(in_features=200, out_features=10, bias=True)
)

In [38]:
optimizer2 = optim.SGD(net2.parameters(), lr=0.01, momentum=0.9)

In [46]:
log_interval = 100
start = time.time()

for epoch in range(1, 10): ## run the model for 10 epochs

    for batch_idx, (data, target) in enumerate(train_loader):
       
        data, target = Variable(data), Variable(target)
        
        #cast to gpu
        data = data.to(device)
        target = target.to(device)
        
        # resize data from (batch_size, 1, 28, 28) to (batch_size, 28*28)
        data = data.view(-1, 28*28)
        #print(data.size())
        
        #reset gradients
        optimizer2.zero_grad()      
        
        ## 1. forward propagation
        net_out = net2(data)

        ## 2. loss calculation
        loss = criterion(net_out, target)
       
        ## 3. backward propagation
        loss.backward()
        
        ## 4. weight optimization
        optimizer2.step()
        

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, 
                batch_idx * len(data), 
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader), 
                loss.data))
            
end = time.time()
print(end-start)

/home/werner/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.297508
Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.299280
Train Epoch: 1 [40000/60000 (67%)]	Loss: 2.300821
Train Epoch: 2 [0/60000 (0%)]	Loss: 2.301765
Train Epoch: 2 [20000/60000 (33%)]	Loss: 2.300425
Train Epoch: 2 [40000/60000 (67%)]	Loss: 2.303541
Train Epoch: 3 [0/60000 (0%)]	Loss: 2.299265
Train Epoch: 3 [20000/60000 (33%)]	Loss: 2.303730
Train Epoch: 3 [40000/60000 (67%)]	Loss: 2.297861
Train Epoch: 4 [0/60000 (0%)]	Loss: 2.294935
Train Epoch: 4 [20000/60000 (33%)]	Loss: 2.303661
Train Epoch: 4 [40000/60000 (67%)]	Loss: 2.298801
Train Epoch: 5 [0/60000 (0%)]	Loss: 2.307049
Train Epoch: 5 [20000/60000 (33%)]	Loss: 2.304901
Train Epoch: 5 [40000/60000 (67%)]	Loss: 2.298889
Train Epoch: 6 [0/60000 (0%)]	Loss: 2.299034
Train Epoch: 6 [20000/60000 (33%)]	Loss: 2.297056
Train Epoch: 6 [40000/60000 (67%)]	Loss: 2.308685
Train Epoch: 7 [0/60000 (0%)]	Loss: 2.298451
Train Epoch: 7 [20000/60000 (33%)]	Loss: 2.301242
Train Epoch: 7 [40000/60000 (67%)]	

In [21]:
torch.cuda.current_device()

0

In [22]:
torch.cuda.device(0)

In [23]:
torch.cuda.device_count()

1

In [24]:
torch.cuda.get_device_name(0)

'GeForce GTX 1060 6GB'

In [25]:
torch.cuda.is_available()

True

In [47]:
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**2,1), 'MB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**2,1), 'MB')

GeForce GTX 1060 6GB
Memory Usage:
Allocated: 12.0 MB
Cached:    32.0 MB
